![](./banner.png)
# Create dataset 1: Concatinate Yellow Card Scheme files

## Overview
Recall from Notebook 1a in the Data Collection process, we downloaded and unzipped all the Yellow Card report files, where we have 3 different `.csv` files for each drug.

The 3 `.csv` files are:
1. `case`: Summary of each report for the drug, includes patient demographics.
2. `event`: Adverse reactions within each report for the drug, there may be multiple events per case.
3. `drug`: Method of administration for the drug in each report.

For this project, I decided to only use `case` and `event` files for each drug, as the `drug` file provides little useful information.

In order to retain each individual side effect incident as a separate data point, I decided to join the `case` file TO the `event` file. This way, I retain the patient-level information, for instance patient gender and age group.

I performed the joining using SQL. Let's first install the libraries and create a connection to a SQLite database.

In [32]:
# import libraries
import pandas as pd
from pandas.io import sql
import sqlite3
from tqdm import tqdm

# create sqlite connection
connection = sqlite3.connect('./yc_db.sqlite')

## Trial with one drug
To make sure I am joining the files the way I intended to, I first trialled with one drug.

### Load csv files
I first loaded in the separate csv files:
> Note `drugs` was not used in the end.

In [9]:
# try with one drug

# events csv to sql
events = pd.read_csv('/Users/JocelynHo/Desktop/yellow_card_raw_csvs/UK_NON_000460896114_event.csv')
events.to_sql(name = 'events', con = connection, if_exists = 'replace', index = False)

# cases csv to sql
reports = pd.read_csv('/Users/JocelynHo/Desktop/yellow_card_raw_csvs/UK_NON_000460896114_case.csv')
reports.to_sql(name = 'reports', con = connection, if_exists = 'replace', index = False)

# drugs csv to sql
drugs = pd.read_csv('/Users/JocelynHo/Desktop/yellow_card_raw_csvs/UK_NON_000460896114_drug.csv')
drugs.to_sql(name = 'drugs', con = connection, if_exists = 'replace', index = False)

print('events:', len(events), 'reports:', len(reports), 'drugs:', len(drugs))

events: 20751 reports: 8145 drugs: 8797


- `Events` has 20751 rows, meaning there are 20751 side effect incidents for this drug.
- `Reports` has 8145 rows, meaning there are 8145 reports submitted to the Yellow Card Scheme for this drug, and in these reports there are 20751 events in total.
- `Drugs` has 8797 rows, meaning for the 8145 reports, there are multiple routes of administration for this drug, resulting in more rows in drugs than in reports.

Let's now look at the individual csv files.

### Events.csv
Here's a brief explanation of what each column means:
- `ADR` - Unique report numbers, corresponds to the ADR numbers in `reports` and `drugs` csv files.
- `SEQ` - Indicates different side effect incidents within each report, for example, the first 3 rows show that ADR 1 has 3 different side effects reported.
- `PT` - Symptom in preferred terms (defined by MedDRA, Medical Dictionary for Regulatory Activities)
- `HLT`,`HLGT` - Symptom in higher level terms
- `SOC_ABBREV` - The system organ class that the symptom belongs to, i.e. category of the side effect.
- `FATAL_YN` - Whether the side effect was fatal or not. Y indicates Yes, N indicates No.

In [90]:
events.head()

,ADR,SEQ,PT,HLT,HLGT,SOC_ABBREV,FATAL_YN
0,1,1,Chills,Feelings and sensations NEC,General system disorders NEC,Genrl,N
1,1,2,Malaise,Asthenic conditions,General system disorders NEC,Genrl,N
2,1,3,Vomiting,Nausea and vomiting symptoms,Gastrointestinal signs and symptoms,Gastr,N
3,2,1,Oral candidiasis,Candida infections,Fungal infectious disorders,Infec,N
4,3,1,Photosensitivity reaction,Photosensitivity and photodermatosis conditions,Epidermal and dermal conditions,Skin,N


In [99]:
# check for any null values
events.isnull().sum()

ADR           0
SEQ           0
PT            0
HLT           0
HLGT          0
SOC_ABBREV    0
FATAL_YN      0
dtype: int64

### Reports.csv

Let's now look at `reports`. Columns include:
- `ADR` - unique number for each report within a drug
- `AGE` - Age group in ranges of 10 years, number shown indicates the lower bound of each 10-year range (eg. 0 = 0 to 10 years old)
- `SENDER_TYPE` - Type of person who sent the report, indirect = from a pharmaceutical company, direct = from a health care professional or the public
- `CONSUMER_YN` - Whether the person involved in the side effect incident is a consumer (patient, parent, carer, lawyer)
- `NONSERIOUS_SERIOUS_FATAL_NSF` - the severity of side effects.

##### Severity of side effects
The severity of a side effect is determined by two factors:

a) As per definitions determined by the MHRA.

b) Whether the original reporter considers it serious.
The term 'serious' is determined by a working group of the Council for International Organizations of Medical Sciences (CIOMS), and is defined as 6 possible categories:
1. patient died due to reaction
2. life threatening
3. resulted in hospitalisation or prolonged inpatient hospitalisation
4. congenital abnormality and
5. involved persistent or significant disability or incapacity, or
6. if the reaction was deemed medically significant.

These are then grouped into the three categories shown: Non-serious, serious, fatal.

In [93]:
reports.head()

,ADR,SEX,AGE_10,RECVD_YEAR,SENDER_TYPE,CONSUMER_YN,HCP_YN,NONSERIOUS_SERIOUS_FATAL_NSF
0,1,Female,40.0,2003,Indirect,N,Y,S
1,2,Female,50.0,2003,Direct,N,Y,N
2,3,Female,60.0,2003,Direct,N,Y,S
3,4,Female,50.0,2003,Direct,N,Y,N
4,5,Female,40.0,2003,Direct,N,Y,N


### Join 2 csvs using SQLite

Now let's try to join `events` and `reports` together.

#### Create custome function for SQL query
First, I created a function that takes in a query string (SQL query in the form of a string of texts), and the connection to the SQLite database we created initially.

In [10]:
# create function to query database
def query(string, connection):
    df = sql.read_sql(string, con = connection)
    return df

#### Left join events and reports
Then, I left joined the two csvs, adding reports to events, to create the data frame for this particular drug.

In [37]:
string = """
SELECT events.ADR as adr, events.SEQ as seq,
        events.PT as side_effect, events.SOC_ABBREV as category,
        reports.SEX as gender, reports.AGE_10 as age,
        reports.RECVD_YEAR as year, reports.SENDER_TYPE as is_direct,
        reports.CONSUMER_YN as is_consumer, reports.HCP_YN as is_hcp,
        reports.NONSERIOUS_SERIOUS_FATAL_NSF as severity
FROM events
LEFT JOIN reports
ON events.ADR = reports.ADR
"""

x = query(string, connection)
x.head()

,adr,seq,side_effect,category,gender,age,year,is_direct,is_consumer,is_hcp,severity
0,1,1,Diabetes mellitus,Metab,Male,50.0,2001,Direct,N,Y,S
1,2,1,Malaise,Genrl,Male,30.0,2001,Direct,N,Y,N
2,2,2,Pyrexia,Genrl,Male,30.0,2001,Direct,N,Y,N
3,2,3,Rash,Skin,Male,30.0,2001,Direct,N,Y,N
4,3,1,Hypersensitivity,Immun,Male,20.0,2001,Indirect,N,Y,N


In [12]:
# check if any rows are duplicated
len(x[x.duplicated()])

0

### Add Yellow Card ID to each drug

Now that I managed to join the two csvs together, the next step is to expand the process for all the drugs in the Yellow Card Scheme data frame.

Before doing that, I decided to add the Yellow Card IDs to each drug after joining the csvs, so that there is an ID to identify which drug causes each side effect.

Since the IDs are in the csv file names, I can simply loop through the different Yellow Card IDs, join the csvs for each ID, and add the Yellow Card ID to the joined data frame as a new column.

In [26]:
x['yc_id'] = '000460896114'
x.head()

,adr,seq,side_effect,category,gender,age,year,is_direct,is_consumer,is_hcp,severity,yc_id
0,1,1,Chills,Genrl,Female,40.0,2003.0,Indirect,N,Y,S,000460896114
1,1,2,Malaise,Genrl,Female,40.0,2003.0,Indirect,N,Y,S,000460896114
2,1,3,Vomiting,Gastr,Female,40.0,2003.0,Indirect,N,Y,S,000460896114
3,2,1,Oral candidiasis,Infec,Female,50.0,2003.0,Direct,N,Y,N,000460896114
4,3,1,Photosensitivity reaction,Skin,Female,60.0,2003.0,Direct,N,Y,S,000460896114


## Automate process for all the drugs
Now that the process works for one drug, let's expand it to all the csvs we downloaded and unzipped from the Yellow Card Scheme website.

### Load Yellow Card IDs
Let's first obtain all the Yellow Card IDs:

In [31]:
# load yellow_card_link csv
db = pd.read_csv('/Users/JocelynHo/Desktop/GA Capstone/drugbank_final.csv',
                      dtype = 'object')

# obtain all yellow card ids
yc_ids = list(db.yc_id.values)

# check number of ids
print(len(yc_ids))

# print out the first 5 ids
yc_ids[:5]

2329


['000040046536',
 '000561378321',
 '000231911819',
 '000369408139',
 '000968368347']

### Repeat process using for loop
Now, I loop through each Yellow Card ID and repeat the entire process:

In [38]:
# loop through each drug
for yc_id in tqdm(yc_ids):
    
    # load in event csv to sql
    events = pd.read_csv(f'/Users/JocelynHo/Desktop/yellow_card_raw_csvs/UK_NON_{yc_id}_event.csv')
    events.to_sql(name = 'events', con = connection, if_exists = 'replace')

    # load in case csv to sql
    reports = pd.read_csv(f'/Users/JocelynHo/Desktop/yellow_card_raw_csvs/UK_NON_{yc_id}_case.csv')
    reports.to_sql(name = 'reports', con = connection, if_exists = 'replace')

    # create SQL query string
    string = """
    SELECT events.ADR as adr, events.SEQ as seq,
            events.PT as side_effect, events.SOC_ABBREV as category,
            reports.SEX as gender, reports.AGE_10 as age,
            reports.RECVD_YEAR as year, reports.SENDER_TYPE as is_direct,
            reports.CONSUMER_YN as is_consumer, reports.HCP_YN as is_hcp,
            reports.NONSERIOUS_SERIOUS_FATAL_NSF as severity
    FROM events
    LEFT JOIN reports
    ON events.ADR = reports.ADR
    """
    
    # use custom function to query database and save combined tables as dataframe
    df = query(string, connection)

    # add yellow card id to every row as new column
    df['yc_id'] = yc_id
    
    # create a new table and save to SQL data base
    df.to_sql(name = 'df', con = connection, if_exists = 'append')

100%|██████████| 2329/2329 [06:37<00:00,  5.86it/s]


#### Double check number of rows
I manually checked the total number of rows in each events csv:

In [41]:
count = 0
for yc_id in tqdm(yc_ids):
    # events csv to sql
    events = pd.read_csv(f'/Users/JocelynHo/Desktop/yellow_card_raw_csvs/UK_NON_{yc_id}_event.csv')
    # count total number of rows from all the events
    count += len(events)
count

100%|██████████| 2329/2329 [00:19<00:00, 120.18it/s]


2426954

And compared it against the database I created to make sure they have the same number of rows:

In [43]:
df = query("SELECT * FROM df", connection)
df.shape

(2426954, 13)

## Overview of data base created 

In [44]:
# first 5 rows of the database I created
df.head()

,index,adr,seq,side_effect,category,gender,age,year,is_direct,is_consumer,is_hcp,severity,yc_id
0,0,1,1,Diabetes mellitus,Metab,Male,50.0,2001.0,Direct,N,Y,S,000040046536
1,1,2,1,Malaise,Genrl,Male,30.0,2001.0,Direct,N,Y,N,000040046536
2,2,2,2,Pyrexia,Genrl,Male,30.0,2001.0,Direct,N,Y,N,000040046536
3,3,2,3,Rash,Skin,Male,30.0,2001.0,Direct,N,Y,N,000040046536
4,4,3,1,Hypersensitivity,Immun,Male,20.0,2001.0,Indirect,N,Y,N,000040046536


In [45]:
# check for null values
df.isnull().sum()

index               0
adr                 0
seq                 0
side_effect         0
category            0
gender            122
age            390027
year              122
is_direct         122
is_consumer       122
is_hcp            122
severity          122
yc_id               0
dtype: int64

At first glance, `age` column appears to be missing a lot of values. Data cleaning process will be discussed in the next notebook.

### Convert Yellow Card ID to DrugBank ID

In order to have a common ID to join all the data frames (Yellow Card, DrugBank, Drugs.com reviews) together, I decided to convert the Yellow Card IDs to DrugBank IDs.

#### Create a dictionary of Yellow Card IDs to DrugBank IDs
First, I loaded in the data I created in the first Notebook 1a. Then, I created a dictionary that maps each Yellow Card ID to their respective DrugBank ID.

In [46]:
# create dictionary
drug_ids = {db['yc_id'].loc[i]: db['db_id'].loc[i] for i in range(len(db))}

# show first 5 items in dictionary
import itertools
dict(itertools.islice(drug_ids.items(), 5))

{'000040046536': 'DB01048',
 '000561378321': 'DB01281',
 '000231911819': 'DB00054',
 '000369408139': 'DB12001',
 '000968368347': 'DB05812'}

#### Create new column for DrugBank ID
Then, I created a new column that contains the DrugBank ID of each drug.

In [47]:
df['db_id'] = df['yc_id'].map(drug_ids)
df.head()

,index,adr,seq,side_effect,category,gender,age,year,is_direct,is_consumer,is_hcp,severity,yc_id,db_id
0,0,1,1,Diabetes mellitus,Metab,Male,50.0,2001.0,Direct,N,Y,S,000040046536,DB01048
1,1,2,1,Malaise,Genrl,Male,30.0,2001.0,Direct,N,Y,N,000040046536,DB01048
2,2,2,2,Pyrexia,Genrl,Male,30.0,2001.0,Direct,N,Y,N,000040046536,DB01048
3,3,2,3,Rash,Skin,Male,30.0,2001.0,Direct,N,Y,N,000040046536,DB01048
4,4,3,1,Hypersensitivity,Immun,Male,20.0,2001.0,Indirect,N,Y,N,000040046536,DB01048


In [48]:
## save as csv
# df.to_csv('df_yc_all.csv', index = False, header = True)

## Next Step:
The next notebook describes steps I took for cleaning this data frame, and adding in data from DrugBank and reviews from Drugs.com.